In [1]:
from sklearn.metrics import classification_report
import numpy as np

In [40]:
import pandas as pd
df=messages = pd.read_csv('df_survey_bi.csv')
df.head()

,text,class
0,so jew hating muslim of them those sneaky jew...,hate
1,Stoni is a fuckin queer...,hate
2,they are out smarting u . anither proof had th...,hate
3,if the people of the state don't want faggots ...,hate
4,Soros needs to die like a diseased dog in the ...,hate


In [ ]:
#hate=0 not_hate=1

In [59]:
df = df.dropna()

In [60]:
df.shape

(899, 2)

In [61]:
X=list(df['text'])

In [62]:
y=list(df['class'])

In [64]:
y=list(pd.get_dummies(y,drop_first=True)['not_hate'])

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [66]:
X_train

['RT @SenorSteez: Man, now these hoes know &#128514;&#128176;&#128182;&#128184; #TheJigIsUp http://t.co/8W9Ag3bx1a',
 'again  prove your claim.are you vile people now claiming stalin was a jew ?',
 'michelle obama goes on racist tirade against white men at womens conference in pennsylvania',
 'Stoni is a fuckin queer...',
 'Will you sell that pussy for me ?',
 "@WaTch_Mark @WaTch_Scenario Normally I would say she must have a great pussy but since I've seen it, that's not true so idk what it is.",
 ' the  " white house " " and " " the white resister " " united with ukraine against russia ... how adolf hitler . * lol * "',
 'the whole abrahamic systems is built on hate  jews  chrstians  & muslims must kill the innocent to',
 "California wet dream: \xa0Rip it's star from our flag. \xa0Attach thousands of boats to the coastline, wrench it from the mainland, tow it out to sea, and sink it with a all aboard.  #Gab #GabFam #SpeakFreely #Chat #Americans",
 'Ask the governor how we are supposed

In [68]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [69]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [70]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [71]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 10,
)

In [72]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [73]:
trainer.evaluate(test_dataset)

{'eval_loss': 2.2214487393697104}

In [74]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 0.823279  , -0.9276675 ],
       [ 3.0791073 , -3.17101   ],
       [ 3.1875167 , -3.3137376 ],
       [ 3.2133684 , -3.3527057 ],
       [ 3.0707617 , -3.2159698 ],
       [ 1.383025  , -1.3558865 ],
       [ 2.794394  , -2.8224933 ],
       [ 3.1020098 , -3.1937616 ],
       [ 3.2223802 , -3.3792634 ],
       [ 3.1389365 , -3.2997017 ],
       [ 2.9502559 , -3.001472  ],
       [ 3.0540483 , -3.1324098 ],
       [ 3.256339  , -3.3378472 ],
       [-0.9902859 ,  0.9120035 ],
       [-2.461459  ,  2.3531122 ],
       [ 1.0095294 , -1.0233293 ],
       [ 0.8771881 , -0.8092537 ],
       [ 0.6596752 , -0.70178324],
       [ 1.2722586 , -1.2225887 ],
       [ 3.0547996 , -3.170204  ],
       [ 3.2614799 , -3.4031584 ],
       [ 1.4682984 , -1.3752345 ],
       [ 3.2577543 , -3.3520086 ],
       [ 1.8970634 , -1.93146   ],
       [ 2.5080168 , -2.5422907 ],
       [ 1.6296079 , -1.6477215 ],
       [ 1.3907447 , -1.3592914 ],
       [ 2.8767338 , -2.95

In [82]:
trainer.predict(test_dataset)[1].shape

(180,)

In [76]:
output=trainer.predict(test_dataset)[1]

In [77]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[83,  0],
       [ 0, 97]], dtype=int64)

In [78]:
trainer.save_model('senti_model')

In [79]:
# model's performance
print(classification_report(y_test, output))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        83
           1       1.00      1.00      1.00        97

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180



In [83]:
X_test_df= X_test

["There are a lot of examples of politicizing sports, both much more old and much more recent. The latest I'm aware of is all this drama about how American football players should or shouldn't behave when the national anthem is playing. Point is, sports is politics, and it's not possible to separate one from another.",
 ' i support slavery so much i think it should be brought back .  " people " " worked and were well behaved back then . "',
 '@IdiotOlympics @ForTheMen fuck her right in the pussy',
 'ban that pos plague called islam ! ! ! they are illegal here ! ! !',
 '@NoMeatballs @mtaraban is so excited about his red velvet brownie.',
 'The idea blue America needs red America is absolutely false. California needs to leave the US immediately. Texas should leave at the same time. The relationship with Washington D.C. is toxic. #ToxicRelationship',
 "Shouldn't he be buried in Vietnam since he was their greatest war hero? ",
 'You bitch like to text emojis',
 "RT @SomeoneBelow: The perso